In [1]:
import findspark
findspark.init("/opt/spark")

from pyspark import SparkContext
import findspark
import pyspark
import random
import numpy as np
from PIL import Image
from pyspark.sql import SparkSession 
import sys

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

22/06/30 04:41:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.csv('./iris.csv', header=True, inferSchema=True)

In [4]:
df.printSchema()

root
 |-- sepalLength: double (nullable = true)
 |-- sepalWidth: double (nullable = true)
 |-- petalLength: double (nullable = true)
 |-- petalWidth: double (nullable = true)
 |-- variety: string (nullable = true)



In [12]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [6]:
input_cols = df.columns

In [7]:
input_cols

['sepalLength', 'sepalWidth', 'petalLength', 'petalWidth', 'variety']

In [8]:
vec_assembler = VectorAssembler(inputCols=input_cols[:-1], outputCol="features")

In [9]:
final_data = vec_assembler.transform(df)

In [15]:
kmeans = KMeans(k=3,featuresCol="features")

In [16]:
model = kmeans.fit(final_data)

In [18]:
model

KMeansModel: uid=KMeans_bea20945bdd5, k=3, distanceMeasure=euclidean, numFeatures=4

In [17]:
model.transform(final_data).groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   50|
|         2|   61|
|         0|   39|
+----------+-----+



In [19]:
predictions = model.transform(final_data)
predictions.show()

+-----------+----------+-----------+----------+-------+-----------------+----------+
|sepalLength|sepalWidth|petalLength|petalWidth|variety|         features|prediction|
+-----------+----------+-----------+----------+-------+-----------------+----------+
|        5.1|       3.5|        1.4|       0.2| Setosa|[5.1,3.5,1.4,0.2]|         1|
|        4.9|       3.0|        1.4|       0.2| Setosa|[4.9,3.0,1.4,0.2]|         1|
|        4.7|       3.2|        1.3|       0.2| Setosa|[4.7,3.2,1.3,0.2]|         1|
|        4.6|       3.1|        1.5|       0.2| Setosa|[4.6,3.1,1.5,0.2]|         1|
|        5.0|       3.6|        1.4|       0.2| Setosa|[5.0,3.6,1.4,0.2]|         1|
|        5.4|       3.9|        1.7|       0.4| Setosa|[5.4,3.9,1.7,0.4]|         1|
|        4.6|       3.4|        1.4|       0.3| Setosa|[4.6,3.4,1.4,0.3]|         1|
|        5.0|       3.4|        1.5|       0.2| Setosa|[5.0,3.4,1.5,0.2]|         1|
|        4.4|       2.9|        1.4|       0.2| Setosa|[4.4,2.9,1

In [21]:
predictions.groupBy("variety","prediction").count().show()

+----------+----------+-----+
|   variety|prediction|count|
+----------+----------+-----+
|Versicolor|         0|    3|
|Versicolor|         2|   47|
| Virginica|         2|   14|
|    Setosa|         1|   50|
| Virginica|         0|   36|
+----------+----------+-----+

